In [65]:
import pandas as pd
import numpy as np
import plotly.express as px
from sentence_transformers import SentenceTransformer
#umap 
import umap
import hdbscan
import plotly.graph_objects as go

In [186]:
#read the data
df = pd.read_csv("data.csv")
df.head()

,content,sentiment
0,"tolong kalo mau maintenance jangan jam segini,...",negatif
1,Min bpjs lagi ga bisa ya skrng,negatif
2,min tolong cek DM ya,netral
3,check dm min.,netral
4,"Saat melakukan pemidahan faskes, apa bisa lang...",netral


In [14]:
df_pos = df[df['sentiment'] == 'positif']
df_neg = df[df['sentiment'] == 'negatif']
df_neu = df[df['sentiment'] == 'netral'] 

### embedding model

In [169]:
embedding_model = SentenceTransformer("firqaaa/indo-sentence-bert-base")

In [170]:
embedding_model.save("sentence_bert")

In [29]:
neu_text_embed =embedding_model.encode(df_neu['content'].values.tolist(), show_progress_bar=True)

Batches: 100%|██████████| 3/3 [00:08<00:00,  2.70s/it]


### UMAP

In [32]:
umap_model = umap.UMAP(n_neighbors=5, min_dist=0.3, metric='cosine') 

### Functions to embed text and plot the embeddings


In [188]:
def plot_text(df,kelas,embedding_model):
    df = df[df.sentiment == kelas]
    data = embedding_model.encode(df.values.tolist())
    umap_model = umap.UMAP(n_neighbors=min(df.shape[0],5),random_state = 42) 
    umap_data = umap_model.fit_transform(data)
    clusterer = hdbscan.HDBSCAN(min_cluster_size=2)
    clusterer.fit(umap_data)

    labels = ['cluster ' + str(i) for i in clusterer.labels_]
    text = df["content"].str.wrap(50).apply(lambda x: x.replace('\n', '<br>'))
    
    fig = px.scatter(x=umap_data[:,0], y=umap_data[:,1],color = clusterer.labels_)
    # remove legend
    fig = px.scatter(x=umap_data[:,0], y=umap_data[:,1],color = labels,text = text)
    #set text color 
    fig.update_traces(textfont_color='rgba(0,0,0,0)',marker_size = 8)
    # remove legend
    fig.update_layout(showlegend=False)
    # set legend title to cluster
    return labels,fig

In [185]:
df = df[df.sentiment == "netral"]
df

,content,sentiment


In [179]:
data = embedding_model.encode(df.values.tolist())

In [184]:
min(df.shape[0]-1,5)

2

In [183]:
umap_model = umap.UMAP(n_neighbors=min(df.shape[0]-1,5),random_state = 42) 
umap_data = umap_model.fit_transform(data)

e:\Anaconda\envs\sentiment_app\lib\site-packages\scipy\sparse\linalg\_eigen\arpack\arpack.py:1592: RuntimeWarning:

k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.



TypeError: Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.

In [189]:
labels, fig = plot_text(df,"netral",embedding_model)

In [190]:
fig.show()

In [159]:
def plot_text(df,embedding_model):
    data = embedding_model.encode(df.values.tolist())
    umap_model = umap.UMAP(n_neighbors=min(len(data),5),random_state = 42) 
    umap_data = umap_model.fit_transform(data)
    clusterer = hdbscan.HDBSCAN(min_cluster_size=3)
    clusterer.fit(umap_data)

    labels = ['cluster ' + str(i) for i in clusterer.labels_]
    text = df.str.wrap(50).apply(lambda x: x.replace('\n', '<br>'))
    
    fig = px.scatter(x=umap_data[:,0], y=umap_data[:,1],color = clusterer.labels_)
    # remove legend
    fig = px.scatter(x=umap_data[:,0], y=umap_data[:,1],color = labels,text = text)
    #set text color 
    fig.update_traces(textfont_color='rgba(0,0,0,0)',marker_size = 8)
    # remove legend
    fig.update_layout(showlegend=False)
    # set legend title to cluster
    fig.show()
    return labels


In [165]:
labels = plot_text(df_neu["content"],embedding_model)

In [151]:
labels = clusterer.labels_
# set labels to string format cluster_1, cluster_2, etc
labels = ['cluster ' + str(i) for i in labels]

In [152]:
text = df_neg['content'].str.wrap(50).apply(lambda x: x.replace('\n', '<br>'))

In [191]:
fig = px.scatter(x=umap_data[:,0], y=umap_data[:,1],color = labels,text = text)
#set text color 
fig.update_traces(textfont_color='rgba(0,0,0,0)',marker_size = 8)
# remove legend
fig.update_layout(showlegend=False)
# set legend title to cluster
# set title text clustering 
fig.update_layout(title_text='Text Clustering')


fig.show()


ValueError: All arguments should have the same length. The length of argument `color` is 72, whereas the length of  previously-processed arguments ['x', 'y', 'text'] is 26

In [136]:
labels

['cluster_0',
 'cluster_3',
 'cluster_6',
 'cluster_6',
 'cluster_1',
 'cluster_6',
 'cluster_0',
 'cluster_2',
 'cluster_1',
 'cluster_4',
 'cluster_4',
 'cluster_1',
 'cluster_-1',
 'cluster_3',
 'cluster_0',
 'cluster_0',
 'cluster_1',
 'cluster_0',
 'cluster_-1',
 'cluster_3',
 'cluster_3',
 'cluster_2',
 'cluster_3',
 'cluster_4',
 'cluster_5',
 'cluster_5']